## Neural Machine Translation with Attention for SQuAD Dataset

In this notebook, Neural Machine Translation with Attention is applied on SQUAD dataset (Standford Question Answering Dataset)

https://rajpurkar.github.io/SQuAD-explorer/




In [ ]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

--2021-01-06 06:40:26--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘squad/train-v2.0.json’

squad/train-v2.0.js 100%[===================>]  40.17M   138MB/s    in 0.3s    

2021-01-06 06:40:28 (138 MB/s) - ‘squad/train-v2.0.json’ saved [42123633/42123633]

--2021-01-06 06:40:28--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘squad/dev-v2.0.json’

squ

In [ ]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
import pandas as pd

train_df = pd.DataFrame(list(zip(train_contexts, train_questions, train_answers)),  columns =['contexts', 'questions', 'answers']) 
valid_df = pd.DataFrame(list(zip(val_contexts, val_questions, val_answers)),  columns =['contexts', 'questions', 'answers']) 

In [ ]:
#train_df['answers'] = train_df['answers'].apply(lambda ans: (json.loads(ans))['text'])
#valid_df['answers'] = valid_df['answers'].apply(lambda ans: (json.loads(ans))['text'])
train_df['answers'] = train_df['answers'].apply(lambda ans: ans['text'])
valid_df['answers'] = valid_df['answers'].apply(lambda ans: ans['text'])

In [ ]:
train_df.head(10)

,contexts,questions,answers
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s
5,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what R&B group was she the lead singer?,Destiny's Child
6,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,Dangerously in Love
7,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,Mathew Knowles
8,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,late 1990s
9,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,lead singer


In [ ]:
import nltk
import string
import re

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer




from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import string
import nltk
import random
import random
#import google_trans_new
#from google_trans_new import google_translator

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
lem = WordNetLemmatizer()
#translator = google_translator()

def clean_text(text):
    ## lower case
    if not isinstance(text, str):
      return str(text) 
    cleaned = text.lower()

    urls_pattern = re.compile(r'https?://\S+|www.\S+')
    cleaned = urls_pattern.sub(r'',cleaned)
    
    ## remove punctuations
    punctuations = string.punctuation
    cleaned_temp = "".join(character for character in cleaned if character not in punctuations)
    
    ## remove stopwords 
    words = cleaned_temp.split()
    #stopword_lists = stopwords.words("english")
    #cleaned = [word for word in words if word not in stopword_lists]
    cleaned = words
    
    ## normalization - lemmatization
    cleaned = [lem.lemmatize(word, "v") for word in cleaned]
    cleaned = [lem.lemmatize(word, "n") for word in cleaned]
    
    ## join 
    cleaned = " ".join(cleaned)
    return cleaned

def back_translate(sentence):
  
  available_langs = list(google_trans_new.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {google_trans_new.LANGUAGES[trans_lang]}")
  translations = translator.translate(sentence, lang_tgt=trans_lang) 
  #print(translations)

  translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
  # print(translations_en_random)
  return translations_en_random

def random_deletion(sentence, p=0.5): 
    words = sentence.split()
    ret_val = ""
    if len(words) == 1: # return if single word
        ret_val = words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        ret_val = [random.choice(words)] 
    else:
        ret_val = remaining
    return " ".join(ret_val)

def random_swap(sentence, n=5):
    words =  sentence.split()
    length = range(len(words)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        words[idx1], words[idx2] = words[idx2], words[idx1] 
    return " ".join(words)

import re
import random
import time
# Replace anonymized screen names with common token @__sn__
def sn_replace(match):
    _sn = match.group(2).lower()
    if not _sn.isnumeric():
        # This is a company screen name
        return match.group(1) + match.group(2)
    return '@__sn__'

sn_re = re.compile('(\W@|^@)([a-zA-Z0-9_]+)')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from tqdm import tqdm_notebook as tqdm
tqdm().pandas() 

train_df['contexts'] = train_df['contexts'].progress_apply(lambda txt: clean_text(txt))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
%%bash
python -m spacy download en


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


**Restart Notebook**

# 3 - Neural Machine Translation by Jointly Learning to Align and Translate

In this third notebook on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473). This model achives our best perplexity yet, ~27 compared to ~34 for the previous model.

## Introduction

As a reminder, here is the general encoder-decoder model:

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq1.png?raw=1)

In the previous model, our architecture was set-up in a way to reduce "information compression" by explicitly passing the context vector, $z$, to the decoder at every time-step and by passing both the context vector and embedded input word, $d(y_t)$, along with the hidden state, $s_t$, to the linear layer, $f$, to make a prediction.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq7.png?raw=1)

Even though we have reduced some of this compression, our context vector still needs to contain all of the information about the source sentence. The model implemented in this notebook avoids this compression by allowing the decoder to look at the entire source sentence (via its hidden states) at each decoding step! How does it do this? It uses *attention*. 

Attention works by first, calculating an attention vector, $a$, that is the length of the source sentence. The attention vector has the property that each element is between 0 and 1, and the entire vector sums to 1. We then calculate a weighted sum of our source sentence hidden states, $H$, to get a weighted source vector, $w$. 

$$w = \sum_{i}a_ih_i$$

We calculate a new weighted source vector every time-step when decoding, using it as input to our decoder RNN as well as the linear layer to make a prediction. We'll explain how to do all of this during the session.

## Preparing Data

Again, the preparation is similar to last time.

First we import all the required modules.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

import torchtext
from torchtext import data 

Set the random seeds for reproducability.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Load the German and English spaCy models.

In [ ]:
spacy_en = spacy.load('en')

We create the tokenizers.

In [ ]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

The fields remain the same as before.

In [ ]:
CONTEXT = Field(tokenize=tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

QUESTION = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

ANSWER = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

Load the data.

In [ ]:
fields = [('context', CONTEXT),('question',QUESTION), ('answer',ANSWER)]

In [ ]:
train_field_value_list = list(zip(train_df['contexts'], train_df['questions'],train_df['answers'] ))
valid_field_value_list = list(zip(valid_df['contexts'], valid_df['questions'],valid_df['answers'] ))

In [ ]:
train_example = [data.Example.fromlist([SRC_TRG[0], SRC_TRG[1], SRC_TRG[2] ], fields ) for SRC_TRG in train_field_value_list if len(SRC_TRG[0]) > 0 and len(SRC_TRG[1]) > 0 and len(SRC_TRG[2]) > 0 ] 

In [ ]:
valid_example = [data.Example.fromlist([SRC_TRG[0], SRC_TRG[1], SRC_TRG[2] ], fields ) for SRC_TRG in valid_field_value_list if len(SRC_TRG[0]) > 0 and len(SRC_TRG[1]) > 0 and len(SRC_TRG[2]) > 0 ] 

In [ ]:
#from sklearn.model_selection import train_test_split
#test_val_split = 0.3
#train_idx, test_val_idx = train_test_split(list(range(len(train_example) )), test_size=test_val_split)

In [ ]:
train_size = int(len(train_example))
train_idx = list(range(train_size))


In [ ]:
val_size = int(0.5* len(valid_example))
valid_example_index_list = list(range(len(valid_example)))
val_idx = valid_example_index_list[0:val_size]
test_idx = valid_example_index_list[val_size:]

In [ ]:
train_list = [train_example[i] for i in train_idx ]
val_list = [valid_example[i] for i in val_idx ]
test_list = [valid_example[i] for i in test_idx ]

In [ ]:
train = data.Dataset(train_list, fields)
valid = data.Dataset(val_list, fields)
test = data.Dataset(test_list, fields)

Build the vocabulary.

In [ ]:
print(vars(train.examples[0]))

{'context': ['beyoncé', 'giselle', 'knowlescarter', 'biːˈjɒnseɪ', 'beeyonsay', 'bear', 'september', '4', '1981', 'be', 'an', 'american', 'singer', 'songwriter', 'record', 'producer', 'and', 'actress', 'bear', 'and', 'raise', 'in', 'houston', 'texas', 'she', 'perform', 'in', 'various', 'sing', 'and', 'dance', 'competition', 'a', 'a', 'child', 'and', 'rise', 'to', 'fame', 'in', 'the', 'late', '1990s', 'a', 'lead', 'singer', 'of', 'rb', 'girlgroup', 'destiny', 'child', 'manage', 'by', 'her', 'father', 'mathew', 'knowles', 'the', 'group', 'become', 'one', 'of', 'the', 'world', 'bestselling', 'girl', 'group', 'of', 'all', 'time', 'their', 'hiatus', 'saw', 'the', 'release', 'of', 'beyoncés', 'debut', 'album', 'dangerously', 'in', 'love', '2003', 'which', 'establish', 'her', 'a', 'a', 'solo', 'artist', 'worldwide', 'earn', 'five', 'grammy', 'award', 'and', 'feature', 'the', 'billboard', 'hot', '100', 'numberone', 'single', 'crazy', 'in', 'love', 'and', 'baby', 'boy'], 'question': ['when', 'di

In [ ]:
MAX_VOCAB_SIZE = 25_000

CONTEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 #vectors = "glove.6B.100d", 
                 vectors = "glove.840B.300d",
                 unk_init = torch.Tensor.normal_)

QUESTION.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 #vectors = "glove.6B.100d", 
                 vectors = "glove.840B.300d",
                 unk_init = torch.Tensor.normal_)

ANSWER.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 #vectors = "glove.6B.100d", 
                 vectors = "glove.840B.300d",
                 unk_init = torch.Tensor.normal_)

.vector_cache/glove.840B.300d.zip: 2.18GB [16:58, 2.14MB/s]                           
100%|█████████▉| 2195599/2196017 [04:10<00:00, 9320.00it/s]

Define the device.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Create the iterators.

In [ ]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train, valid, test), batch_size = BATCH_SIZE, 
                                                           sort_key = lambda x: len(x.context),
                                                           sort_within_batch=True, device = device)

In [ ]:
for obj in valid_iterator:
  print(obj)


[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 41x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 28x64 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 7x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 52x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 28x64 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 8x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 57x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 24x64 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 10x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 62x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 18x64 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 6x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 63x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 

In [ ]:
for obj in train_iterator:
  print(obj)

Streaming output truncated to the last 5000 lines.

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 102x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 23x64 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 19x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 114x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 26x64 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 28x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 123x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 24x64 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 27x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 93x64 (GPU 0)]
	[.question]:[torch.cuda.LongTensor of size 27x64 (GPU 0)]
	[.answer]:[torch.cuda.LongTensor of size 20x64 (GPU 0)]

[torchtext.data.batch.Batch of size 64]
	[.context]:[torch.cuda.LongTensor of size 14

## Building the Seq2Seq Model

### Encoder

First, we'll build the encoder. Similar to the previous model, we only use a single layer GRU, however we now use a *bidirectional RNN*. With a bidirectional RNN, we have two RNNs in each layer. A *forward RNN* going over the embedded sentence from left to right (shown below in green), and a *backward RNN* going over the embedded sentence from right to left (teal). All we need to do in code is set `bidirectional = True` and then pass the embedded sentence to the RNN as before. 

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq8.png?raw=1)

We now have:

$$\begin{align*}
h_t^\rightarrow &= \text{EncoderGRU}^\rightarrow(e(x_t^\rightarrow),h_{t-1}^\rightarrow)\\
h_t^\leftarrow &= \text{EncoderGRU}^\leftarrow(e(x_t^\leftarrow),h_{t-1}^\leftarrow)
\end{align*}$$

Where $x_0^\rightarrow = \text{<sos>}, x_1^\rightarrow = \text{guten}$ and $x_0^\leftarrow = \text{<eos>}, x_1^\leftarrow = \text{morgen}$.

As before, we only pass an input (`embedded`) to the RNN, which tells PyTorch to initialize both the forward and backward initial hidden states ($h_0^\rightarrow$ and $h_0^\leftarrow$, respectively) to a tensor of all zeros. We'll also get two context vectors, one from the forward RNN after it has seen the final word in the sentence, $z^\rightarrow=h_T^\rightarrow$, and one from the backward RNN after it has seen the first word in the sentence, $z^\leftarrow=h_T^\leftarrow$.

The RNN returns `outputs` and `hidden`. 

`outputs` is of size **[src len, batch size, hid dim * num directions]** where the first `hid_dim` elements in the third axis are the hidden states from the top layer forward RNN, and the last `hid_dim` elements are hidden states from the top layer backward RNN. We can think of the third axis as being the forward and backward hidden states concatenated together other, i.e. $h_1 = [h_1^\rightarrow; h_{T}^\leftarrow]$, $h_2 = [h_2^\rightarrow; h_{T-1}^\leftarrow]$ and we can denote all encoder hidden states (forward and backwards concatenated together) as $H=\{ h_1, h_2, ..., h_T\}$.

`hidden` is of size **[n layers * num directions, batch size, hid dim]**, where **[-2, :, :]** gives the top layer forward RNN hidden state after the final time-step (i.e. after it has seen the last word in the sentence) and **[-1, :, :]** gives the top layer backward RNN hidden state after the final time-step (i.e. after it has seen the first word in the sentence).

As the decoder is not bidirectional, it only needs a single context vector, $z$, to use as its initial hidden state, $s_0$, and we currently have two, a forward and a backward one ($z^\rightarrow=h_T^\rightarrow$ and $z^\leftarrow=h_T^\leftarrow$, respectively). We solve this by concatenating the two context vectors together, passing them through a linear layer, $g$, and applying the $\tanh$ activation function. 

$$z=\tanh(g(h_T^\rightarrow, h_T^\leftarrow)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$$

**Note**: this is actually a deviation from the paper. Instead, they feed only the first backward RNN hidden state through a linear layer to get the context vector/decoder initial hidden state. This doesn't seem to make sense to me, so we have changed it.

As we want our model to look back over the whole of the source sentence we return `outputs`, the stacked forward and backward hidden states for every token in the source sentence. We also return `hidden`, which acts as our initial hidden state in the decoder.

In [ ]:
class Encoder(nn.Module):
    def __init__(self, context_input_dim, context_emb_dim, context_enc_hid_dim, question_input_dim, question_emb_dim, question_enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.context_embedding = nn.Embedding(context_input_dim, context_emb_dim)
        
        self.context_rnn = nn.GRU(context_emb_dim, context_enc_hid_dim, bidirectional = True)
        
        self.context_fc = nn.Linear(context_enc_hid_dim * 2, dec_hid_dim)

        self.question_embedding = nn.Embedding(question_input_dim, question_emb_dim)
        
        self.question_rnn = nn.GRU(question_emb_dim,question_enc_hid_dim, bidirectional = True)
        
        self.question_fc = nn.Linear(question_enc_hid_dim * 2, dec_hid_dim)

        self.fc = nn.Linear(dec_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, context, question):
        
        #src = [src len, batch size]
        
        context_embedded = self.dropout(self.context_embedding(context))
        
        #embedded = [src len, batch size, emb dim]
        
        context_outputs, context_hidden = self.context_rnn(context_embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        #context_hidden = torch.tanh(self.fc(torch.cat((context_hidden[-2,:,:], context_hidden[-1,:,:]), dim = 1)))
        context_hidden = self.context_fc(torch.cat((context_hidden[-2,:,:], context_hidden[-1,:,:]), dim = 1))


        question_embedded = self.dropout(self.question_embedding(question))
        
        #embedded = [src len, batch size, emb dim]
        
        question_outputs, question_hidden = self.question_rnn(question_embedded )
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        #context_hidden = torch.tanh(self.fc(torch.cat((context_hidden[-2,:,:], context_hidden[-1,:,:]), dim = 1)))
        question_hidden = self.question_fc(torch.cat((question_hidden[-2,:,:], question_hidden[-1,:,:]), dim = 1))
        hidden = torch.tanh(self.fc(torch.cat((context_hidden, question_hidden), dim = 1)))

        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return context_outputs, hidden

### Attention

Next up is the attention layer. This will take in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.

Intuitively, this layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$. 

First, we calculate the *energy* between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is `repeat` the previous decoder hidden state $T$ times. We then calculate the energy, $E_t$, between them by concatenating them together and passing them through a linear layer (`attn`) and a $\tanh$ activation function. 

$$E_t = \tanh(\text{attn}(s_{t-1}, H))$$ 

This can be thought of as calculating how well each encoder hidden state "matches" the previous decoder hidden state.

We currently have a **[dec hid dim, src len]** tensor for each example in the batch. We want this to be **[src len]** for each example in the batch as the attention should be over the length of the source sentence. This is achieved by multiplying the `energy` by a **[1, dec hid dim]** tensor, $v$.

$$\hat{a}_t = v E_t$$

We can think of $v$ as the weights for a weighted sum of the energy across all encoder hidden states. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but learned with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. We implement $v$ as a linear layer without a bias.

Finally, we ensure the attention vector fits the constraints of having all elements between 0 and 1 and the vector summing to 1 by passing it through a $\text{softmax}$ layer.

$$a_t = \text{softmax}(\hat{a_t})$$

This gives us the attention over the source sentence!

Graphically, this looks something like below. This is for calculating the very first attention vector, where $s_{t-1} = s_0 = z$. The green/teal blocks represent the hidden states from both the forward and backward RNNs, and the attention computation is all done within the pink block.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq9.png?raw=1)

In [ ]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

### Decoder

Next up is the decoder. 

The decoder contains the attention layer, `attention`, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.

We then use this attention vector to create a weighted source vector, $w_t$, denoted by `weighted`, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.

$$w_t = a_t H$$

The embedded input word, $d(y_t)$, the weighted source vector, $w_t$, and the previous decoder hidden state, $s_{t-1}$, are then all passed into the decoder RNN, with $d(y_t)$ and $w_t$ being concatenated together.

$$s_t = \text{DecoderGRU}(d(y_t), w_t, s_{t-1})$$

We then pass $d(y_t)$, $w_t$ and $s_t$ through the linear layer, $f$, to make a prediction of the next word in the target sentence, $\hat{y}_{t+1}$. This is done by concatenating them all together.

$$\hat{y}_{t+1} = f(d(y_t), w_t, s_t)$$

The image below shows decoding the first word in an example translation.

![](https://github.com/bentrevett/pytorch-seq2seq/blob/master/assets/seq2seq10.png?raw=1)

The green/teal blocks show the forward/backward encoder RNNs which output $H$, the red block shows the context vector, $z = h_T = \tanh(g(h^\rightarrow_T,h^\leftarrow_T)) = \tanh(g(z^\rightarrow, z^\leftarrow)) = s_0$, the blue block shows the decoder RNN which outputs $s_t$, the purple block shows the linear layer, $f$, which outputs $\hat{y}_{t+1}$ and the orange block shows the calculation of the weighted sum over $H$ by $a_t$ and outputs $w_t$. Not shown is the calculation of $a_t$.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

### Seq2Seq

This is the first model where we don't have to have the encoder RNN and decoder RNN have the same hidden dimensions, however the encoder has to be bidirectional. This requirement can be removed by changing all occurences of `enc_dim * 2` to `enc_dim * 2 if encoder_is_bidirectional else enc_dim`. 

This seq2seq encapsulator is similar to the last two. The only difference is that the `encoder` returns both the final hidden state (which is the final hidden state from both the forward and backward encoder RNNs passed through a linear layer) to be used as the initial hidden state for the decoder, as well as every hidden state (which are the forward and backward hidden states stacked on top of each other). We also need to ensure that `hidden` and `encoder_outputs` are passed to the decoder. 

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, context, question, answer, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = answer.shape[1]
        trg_len = answer.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(context, question)
                
        #first input to the decoder is the <sos> tokens
        input = answer[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = answer[t] if teacher_force else top1

        return outputs

## Training the Seq2Seq Model

The rest of this session is very similar to the previous one.

We initialise our parameters, encoder, decoder and seq2seq model (placing it on the GPU if we have one). 

In [ ]:
CONTEXT_INPUT_DIM = len(CONTEXT.vocab)
QUESTION_INPUT_DIM = len(QUESTION.vocab)
OUTPUT_DIM = len(ANSWER.vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(CONTEXT_INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, QUESTION_INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

We use a simplified version of the weight initialization scheme used in the paper. Here, we will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.

In [ ]:
context_pretrained_embeddings = CONTEXT.vocab.vectors
print(context_pretrained_embeddings.shape)

torch.Size([25004, 300])


In [ ]:
enc.context_embedding.weight.data.copy_(context_pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.1020,  1.6282,  2.1635,  ..., -0.6009, -0.1467,  0.0285],
        ...,
        [-0.1059,  0.0421, -0.0415,  ...,  0.7597,  0.8584,  0.1016],
        [ 0.4681,  0.1534, -0.0751,  ...,  0.6741,  0.0302, -0.7942],
        [ 0.2445, -0.0398,  1.0466,  ..., -0.5515,  0.2743,  0.0540]],
       device='cuda:0')

In [ ]:
PAD_IDX = CONTEXT.vocab.stoi[CONTEXT.pad_token]
UNK_IDX = CONTEXT.vocab.stoi[CONTEXT.unk_token]

enc.context_embedding.weight.data[UNK_IDX] = torch.zeros(ENC_EMB_DIM)
enc.context_embedding.weight.data[PAD_IDX] = torch.zeros(ENC_EMB_DIM)


In [ ]:
question_pretrained_embeddings = QUESTION.vocab.vectors
print(question_pretrained_embeddings.shape)

torch.Size([25004, 300])


In [ ]:
enc.question_embedding.weight.data.copy_(question_pretrained_embeddings)

tensor([[ 1.1717, -0.5923,  0.8272,  ..., -0.2469, -1.2527, -0.8069],
        [-0.8039,  0.1055, -1.1249,  ...,  1.1623,  1.1744, -1.4357],
        [-0.0181, -0.7427,  1.2195,  ...,  0.3657, -0.7106,  2.0183],
        ...,
        [-0.3548, -0.2170, -0.1336,  ...,  0.1289,  0.3064,  0.5214],
        [-0.3818,  0.9061,  0.2361,  ...,  0.2307,  0.0679, -0.4013],
        [ 0.0765, -0.0882,  0.6453,  ...,  0.0660,  0.1781, -0.3045]],
       device='cuda:0')

In [ ]:
PAD_IDX = QUESTION.vocab.stoi[QUESTION.pad_token]
UNK_IDX = QUESTION.vocab.stoi[QUESTION.unk_token]

enc.question_embedding.weight.data[UNK_IDX] = torch.zeros(ENC_EMB_DIM)
enc.question_embedding.weight.data[PAD_IDX] = torch.zeros(ENC_EMB_DIM)


In [ ]:
answer_pretrained_embeddings = ANSWER.vocab.vectors
print(answer_pretrained_embeddings.shape)

torch.Size([25004, 300])


In [ ]:
dec.embedding.weight.data.copy_(answer_pretrained_embeddings)

tensor([[-7.1919e-01,  1.4055e+00, -1.1218e+00,  ..., -1.1134e+00,
         -6.1973e-01, -1.2951e-02],
        [-1.2408e+00, -1.6846e+00, -4.0244e-02,  ..., -3.0458e-01,
          4.1338e-01,  2.3290e-01],
        [-2.6093e-01,  5.0085e-01, -1.1767e-01,  ..., -1.9507e+00,
          1.8153e-01, -4.4086e-01],
        ...,
        [-3.1120e-01,  1.0099e-01, -5.7820e-02,  ..., -1.9612e-02,
          3.0050e-01,  4.1436e-01],
        [ 7.3094e-01, -1.5778e-03,  9.0232e-01,  ..., -1.3792e-01,
         -5.3065e-02, -3.5855e-01],
        [-3.9045e-01, -7.7910e-01,  6.2850e-01,  ...,  7.1024e-01,
          4.3324e-01,  1.8231e-01]], device='cuda:0')

In [ ]:
PAD_IDX = ANSWER.vocab.stoi[ANSWER.pad_token]
UNK_IDX = ANSWER.vocab.stoi[ANSWER.unk_token]

dec.embedding.weight.data[UNK_IDX] = torch.zeros(DEC_EMB_DIM)
dec.embedding.weight.data[PAD_IDX] = torch.zeros(DEC_EMB_DIM)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (context_embedding): Embedding(25004, 300)
    (context_rnn): GRU(300, 256, bidirectional=True)
    (context_fc): Linear(in_features=512, out_features=256, bias=True)
    (question_embedding): Embedding(25004, 300)
    (question_rnn): GRU(300, 256, bidirectional=True)
    (question_fc): Linear(in_features=512, out_features=256, bias=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (embedding): Embedding(25004, 300)
    (rnn): GRU(812, 256)
    (fc_out): Linear(in_features=1068, out_features=25004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

Calculate the number of parameters. We get an increase of almost 50% in the amount of parameters from the last model. 

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 52,359,916 trainable parameters


We create an optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=2e-4)

We initialize the loss function.

In [ ]:
TRG_PAD_IDX = ANSWER.vocab.stoi[ANSWER.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        context = batch.context
        question = batch.question
        answer = batch.answer
        
        optimizer.zero_grad()
        
        output = model(context, question, answer)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        answer = answer[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output,answer)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            context = batch.context
            question = batch.question
            answer = batch.answer
        

            output = model(context, question, answer, 0) #turn off teacher forcing


            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            answer = answer[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, answer)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

Finally, define a timing function.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [ ]:
MODEL_PATH = '/content/drive/My Drive/nlp_end/machine_translation_attention_squad.pth'

In [ ]:
N_EPOCHS = 15
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_PATH)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 4m 42s
	Train Loss: 6.571 | Train PPL: 713.983
	 Val. Loss: 5.863 |  Val. PPL: 351.700
Epoch: 02 | Time: 4m 45s
	Train Loss: 6.166 | Train PPL: 476.377
	 Val. Loss: 5.756 |  Val. PPL: 316.160
Epoch: 03 | Time: 4m 42s
	Train Loss: 5.969 | Train PPL: 391.094
	 Val. Loss: 5.751 |  Val. PPL: 314.444
Epoch: 04 | Time: 4m 44s
	Train Loss: 5.814 | Train PPL: 335.070
	 Val. Loss: 5.773 |  Val. PPL: 321.624
Epoch: 05 | Time: 4m 43s
	Train Loss: 5.629 | Train PPL: 278.492
	 Val. Loss: 5.717 |  Val. PPL: 303.999
Epoch: 06 | Time: 4m 43s
	Train Loss: 5.461 | Train PPL: 235.372
	 Val. Loss: 5.684 |  Val. PPL: 294.120
Epoch: 07 | Time: 4m 44s
	Train Loss: 5.303 | Train PPL: 200.863
	 Val. Loss: 5.687 |  Val. PPL: 294.950
Epoch: 08 | Time: 4m 44s
	Train Loss: 5.145 | Train PPL: 171.566
	 Val. Loss: 5.749 |  Val. PPL: 314.025
Epoch: 09 | Time: 4m 43s
	Train Loss: 4.989 | Train PPL: 146.800
	 Val. Loss: 5.776 |  Val. PPL: 322.440
Epoch: 10 | Time: 4m 43s
	Train Loss: 4.847 | Train PPL

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load(MODEL_PATH))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.695 | Test PPL: 297.250 |


We've improved on the previous model, but this came at the cost of doubling the training time.

Next, we'll be using the same architecture but using a few tricks that are applicable to all RNN architectures - packed padded sequences and masking. We'll also implement code which will allow us to look at what words in the input the RNN is paying attention to when decoding the output. Check this [Notebook](https://colab.research.google.com/github/bentrevett/pytorch-seq2seq/blob/master/4%20-%20Packed%20Padded%20Sequences%2C%20Masking%2C%20Inference%20and%20BLEU.ipynb) 